In [1]:
import torch
import numpy as np
import matplotlib.pyplot as plt
from torch.autograd import Variable

import torch.nn as nn
import torch.nn.functional as F

In [2]:
import load_script

In [ ]:
import os,sys,inspect
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
sys.path.insert(0,parentdir) 

import dlc_bci

In [ ]:
train,test=dlc_bci.load('../data')
# test[test==0]=-1


In [ ]:
new_test=torch.ones(test.shape[0],2)

In [ ]:
new_test[:,0][test==1]=-1
new_test[:,1][test==0]=-1

In [ ]:
train_dataset=Variable(train)
test_dataset=Variable(new_test)

In [3]:
train_dataset,target=load_script.load_dataset(train=True)

In [4]:
train_dataset

Variable containing:
( 0 ,.,.) = 
   41.8000   44.8000   47.1000  ...    69.8000   72.6000   76.1000
  -10.3000   -5.9000   -3.3000  ...    12.6000   24.0000   26.5000
   38.1000   25.2000   46.0000  ...    45.1000   74.1000   64.8000
              ...                ⋱                ...             
    7.9000   11.2000   14.3000  ...    32.7000   43.4000   45.5000
   19.2000   33.6000   33.8000  ...    46.7000   53.7000   43.4000
   -0.4000   12.7000   12.0000  ...    30.7000   40.6000   33.1000

( 1 ,.,.) = 
  132.0000  137.0000  148.0000  ...    78.1000   90.5000   85.7000
   13.0000    0.4000   22.9000  ...   -43.6000  -11.0000  -36.6000
   24.3000    5.2000   30.6000  ...   -21.7000    9.9000  -17.3000
              ...                ⋱                ...             
  -23.0000  -19.0000  -25.3000  ...   -30.9000  -19.8000  -16.7000
    0.5000    2.4000   -2.3000  ...    -6.8000   10.0000   16.9000
  -31.8000  -21.5000  -27.8000  ...   -44.6000  -31.7000  -19.1000

( 2 ,.,.) = 


In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net,self).__init__()
        
        
    def forward(self,x):
        pass